# Shunting Yard Algorithm

convert infix to postfix


In [2]:
import re

def solve_or_in_brackets(regex):
    # loop over the string to add | between any two consecutive characters inside square brackets
    i = 0
    regex = list(regex)
    while(i < len(regex)):
        if regex[i] == '[':
            while(regex[i] != ']'):
                if regex[i].isalnum() and regex[i+1].isalnum():
                    regex.insert(i+1, '|')
                i += 1
        else :
            i = i + 1
    return ''.join(regex)


def solve_ranges_in_brackets(regex):
    # loop over the string to add ranges of characters inside square brackets
    i = 0
    regex = list(regex)
    while(i < len(regex)):
        if regex[i] == '[':
            regex[i] = '('
            while(regex[i] != ']'):
                if regex[i] == '-':
                    first = regex[i-1]
                    last = regex[i+1]
                    chars_between = [chr(i) for i in range(ord(first) + 1, ord(last))]
                    str = []
                    for c in chars_between:
                        str.append("|")
                        str.append(c)
                    str.append('|')
                    regex[i:i+1] = str
                    print(''.join(regex))
                i += 1
            regex[i] = ')'
        else :
            i = i + 1
    return ''.join(regex)

def replace_square(regex):
    regex = solve_or_in_brackets(regex)
    regex = solve_ranges_in_brackets(regex)
    return regex

# def replace_square(regex):
#     i = 0
#     regex = list(regex)
#     while(i < len(regex)):
#         if regex[i] == '[':
#             regex[i] = '('
#             while(regex[i] != ']'):
#                 if regex[i] == '-':
#                     first = regex[i-1]
#                     last = regex[i+1]
#                     chars_between = [chr(i) for i in range(ord(first) + 1, ord(last)+1)]
#                     str = []
#                     for c in chars_between:
#                         str.append("|")
#                         str.append(c)
#                     str.append('|')
#                     regex[i:i+2] = str
#                 else : 
#                     # print(regex[i],regex[i+1])
#                     if regex[i] != '(' and regex[i] != '|' and regex[i+1] != '|' and regex[i+1] != '-': 
#                         regex.insert(i+1, '|')
#                 i += 1
#             regex[i-1:i+1] = ')'
#         else :
#             i = i + 1
#     return ''.join(regex)

def replace_dot(regex):
    result = []
    regex = ''.join(regex)
    
    for curr in regex:
        if curr == '.':
            result.append("[a-zA-Z0-9]")
        else : 
            result.append(curr)
    
    return ''.join(result)


def add_concat(regex):
    result = []
    prev = None

    for curr in regex:
        if prev and (prev.isalnum() or prev in ('*', '+', ')','?')) and (curr.isalnum() or curr == '(' ):
            result.append('·')  
        result.append(curr)
        prev = curr
    return ''.join(result)

def preprocessing(regex):

    regex = replace_dot(regex)
    print("\nRegex after replacing dot  : ")
    print(regex)
    regex = replace_square(regex)
    print("\nRegex after replacing square brackets  : ")
    print(regex)
    # regex = add_concat(regex)
    print("\nRegex after adding concatination  : ")
    print(regex)    
    return regex



def infix_to_postfix(infix):
    try:
        re.compile(infix)
    except re.error:
        print("Invalid regex")
        return None
    # Add implicit concatenation operator ('.')
    output = []
    for i, char in enumerate(infix):
        output.append(char)
        
        if i < len(infix) - 1:
            next_char = infix[i + 1]
            if char not in "(|." and next_char not in ")*+?|)":
                output.append('.')
    infix = output

    precedence = { '*' : 5, '+' : 4, '?' : 3 , '.' : 2 , '|' : 1 , '(' : 0 }

    infix = list(infix)
    stack = []
    postfix = []
    for i,char in enumerate(infix):
        
        if char == '(':
            stack.append(char)
        
        elif char == ")":
            stack_top = stack.pop()
            while stack_top != "(":
                postfix.append(stack_top)
                stack_top = stack.pop()
        
        elif char in ['*','+','?','.','|']:
            stack_top = None
            if len(stack) > 0:
                stack_top = stack.pop()
            while stack_top != None  and precedence[char] <= precedence[stack_top]:
                postfix.append(stack_top)
                if len(stack) != 0:
                    stack_top = stack.pop()
                else :
                    break
            if stack_top != None and precedence[char] > precedence[stack_top]:
                stack.append(stack_top)
            stack.append(char)
        
        else:
            postfix.append(char)
                        
    
    while stack:
        top = stack.pop()
        if top == '(':
            print("Unbalanced parentheses")
            return None
        postfix.append(top)
    
    return ''.join(postfix)


infix = "[a-g]?o+"
infix="(a*)*"
preprocessed_infix = preprocessing(infix)
print("\nPOSTFIX :")
print(infix_to_postfix(preprocessed_infix))



Regex after replacing dot  : 
(a*)*

Regex after replacing square brackets  : 
(a*)*

Regex after adding concatination  : 
(a*)*

POSTFIX :
a**


# Postfix to nfa

In [3]:
import json

class NFA:
    def __init__(self, states, starting_state=None, accepting_states=[]):
        self.states = states
        self.starting_state = starting_state
        self.accepting_states = accepting_states

class State1:
    def __init__(self, name, accepting=False):
        self.name = name
        self.transitions = {}  # Dictionary: {symbol: [target_states]}
        self.accepting = accepting

    def add_transition(self, symbol, target_state):
        if symbol not in self.transitions:
            self.transitions[symbol] = []
        self.transitions[symbol].append(target_state)

def new_state(state_counter, accepting=False):
    state = State1(f"S{state_counter}", accepting)
    return state

def handle_character(stack, state_counter, symbol):
    start = new_state(state_counter)
    end = new_state(state_counter + 1, True)
    start.add_transition(symbol, end)
    stack.append(NFA([start, end], start, [end]))
    return state_counter + 2

def handle_concatenation(stack, state_counter):
    if len(stack) < 2:
        raise ValueError("Invalid postfix expression for concatenation")
    nfa2 = stack.pop()
    nfa1 = stack.pop()
    for accepting_state in nfa1.accepting_states:
        accepting_state.accepting = False
        accepting_state.add_transition('ε', nfa2.starting_state)
    combined_nfa = NFA(nfa1.states + nfa2.states, nfa1.starting_state, nfa2.accepting_states)
    stack.append(combined_nfa)
    return combined_nfa,state_counter

def handle_or(stack, state_counter):
    nfa2 = stack.pop()
    nfa1 = stack.pop()
    start = new_state(state_counter)
    end = new_state(state_counter + 1, True)

    start.add_transition('ε', nfa1.starting_state)
    start.add_transition('ε', nfa2.starting_state)

    for accepting_state in nfa1.accepting_states + nfa2.accepting_states:
        accepting_state.accepting = False
        accepting_state.add_transition('ε', end)

    combined_nfa = NFA(nfa1.states + nfa2.states + [start, end], start, [end])
    stack.append(combined_nfa)
    return state_counter + 2

def handle_kleene_star(stack, state_counter):
    nfa = stack.pop()
    start = new_state(state_counter)
    end = new_state(state_counter + 1, True)

    start.add_transition('ε', nfa.starting_state)
    start.add_transition('ε', end)

    for accepting_state in nfa.accepting_states:
        accepting_state.accepting = False
        accepting_state.add_transition('ε', end)
        accepting_state.add_transition('ε', start)

    combined_nfa = NFA(nfa.states + [start, end], start, [end])
    stack.append(combined_nfa)
    return state_counter + 2

def handle_one_or_more(stack, state_counter):
    nfa = stack.pop()
    start = new_state(state_counter)
    end = new_state(state_counter + 1, True)

    start.add_transition('ε', nfa.starting_state)

    for accepting_state in nfa.accepting_states:
        accepting_state.accepting = False
        accepting_state.add_transition('ε', end)
        accepting_state.add_transition('ε', nfa.starting_state)

    combined_nfa = NFA(nfa.states + [start, end], start, [end])
    stack.append(combined_nfa)
    return state_counter + 2

def handle_zero_or_one(stack, state_counter):
    nfa = stack.pop()
    start = new_state(state_counter)
    end = new_state(state_counter + 1, True)

    start.add_transition('ε', nfa.starting_state)
    start.add_transition('ε', end)

    for accepting_state in nfa.accepting_states:
        accepting_state.accepting = False
        accepting_state.add_transition('ε', end)

    combined_nfa = NFA(nfa.states + [start, end], start, [end])
    stack.append(combined_nfa)
    return state_counter + 2

def construct_nfa(postfix_regex):
    state_counter = 1
    stack = []

    for symbol in postfix_regex:
        if symbol.isalnum():
            state_counter = handle_character(stack, state_counter, symbol)
        elif symbol == '.':
            state_counter=handle_concatenation(stack, state_counter)
        elif symbol == '|':
            state_counter = handle_or(stack, state_counter)
        elif symbol == '*':
            state_counter = handle_kleene_star(stack, state_counter)
        elif symbol == '+':
            state_counter = handle_one_or_more(stack, state_counter)
        elif symbol == '?':
            state_counter = handle_zero_or_one(stack, state_counter)
        else:
            raise ValueError(f"Invalid symbol in postfix regex: {symbol}")

    return stack.pop()

def nfa_to_json(nfa):
    nfa_dict = {
        "startingState": nfa.starting_state.name,
    }

    for state in nfa.states:
        transitions = {symbol: [target.name for target in targets] for symbol, targets in state.transitions.items()}
        if state.accepting:
            transitions["isTerminatingState"] = True
        nfa_dict[state.name] = transitions

    return json.dumps(nfa_dict, indent=6, ensure_ascii=False)
def save_nfa_to_file(nfa, filename):
    nfa_json = nfa_to_json(nfa)
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(nfa_json)



# Visualizing NFA

In [4]:
from graphviz import Digraph

def visualize_nfa(nfa):
    dot = Digraph(format='png')
    dot.attr(rankdir='LR')

    # Define the starting state
    dot.node(nfa.starting_state.name, shape='circle', style='filled', fillcolor='lightblue')

    # Define the states
    for state in nfa.states:
        shape = 'doublecircle' if state.accepting else 'circle'
        dot.node(state.name, shape=shape)

    # Add transitions
    for state in nfa.states:
        for symbol, targets in state.transitions.items():
            for target in targets:
                dot.edge(state.name, target.name, label=symbol)

    # Add an initial arrow
    dot.node('start', shape='none', label='')
    dot.edge('start', nfa.starting_state.name)

    # Save and render
    dot.render('nfa_visualization', view=True)




In [5]:
regex="(a*)*"
preprocessed_infix = preprocessing(regex)
print("\nPOSTFIX :")
postfix_regex=infix_to_postfix(preprocessed_infix)
print(postfix_regex)

nfa = construct_nfa(postfix_regex)

visualize_nfa(nfa)

save_nfa_to_file(nfa, "nfa_output.json")




Regex after replacing dot  : 
(a*)*

Regex after replacing square brackets  : 
(a*)*

Regex after adding concatination  : 
(a*)*

POSTFIX :
a**


# Part 2

In [7]:
# from output json to input of part2
def normalize_nfa( NFA):
        normalized_nfa = {}

        for state, transitions in NFA.items():
            if state == "startingState":
                normalized_nfa[state] = transitions
                continue

            normalized_nfa[state] = {}
            for symbol, target in transitions.items():
                if symbol == "isTerminatingState":
                    normalized_nfa[state][symbol] = target
                else:
                    if not isinstance(target, list):
                        target = [target]
                    normalized_nfa[state][symbol] = target

        return normalized_nfa
def read_json_file(file_path):
    try:
        with open(file_path, 'r',encoding='utf-8') as file:
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
    except json.JSONDecodeError:
        print("Error: Failed to decode JSON from the file.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
#read json file in dict
nfa_dict=read_json_file("nfa_output.json")
normalize_nfa(nfa_dict)

{'startingState': 'S5',
 'S1': {'a': ['S2']},
 'S2': {'ε': ['S4', 'S3']},
 'S3': {'ε': ['S1', 'S4']},
 'S4': {'ε': ['S6', 'S5']},
 'S5': {'ε': ['S3', 'S6']},
 'S6': {'isTerminatingState': True}}

In [ ]:
class State:
    def __init__(self, name, elements):
        self.name = name
        self.elements = frozenset(elements)  # Ensure it's hashable

    def __hash__(self):
        return hash(self.elements)

    def __eq__(self, other):
        return self.elements == other.elements

class DFA:
    def __init__(self):
        self.states = {}
        self.startingState = None
        self.transitions = {}
        self.acceptingStates = set()

    def add_state(self, state, is_accepting=False):
        self.states[state.name] = state
        if is_accepting:
            self.acceptingStates.add(state.name)

    def add_transition(self, from_state, to_state, symbol):
        if from_state.name not in self.transitions:
            self.transitions[from_state.name] = {}
        self.transitions[from_state.name][symbol] = to_state.name
    def visualize(self, output_file="dfa"):
        dot = Digraph(format="png")
        dot.attr(rankdir='LR')

        
        # Add states (mark accepting states with double circle)
        for state_name in self.states:
            if state_name in self.acceptingStates:
                dot.node(state_name, shape="doublecircle")
            else:
                dot.node(state_name, shape="circle")

        # Add the starting arrow
        dot.node("start", shape="none", label="")
        dot.edge("start", self.startingState)

        # Add transitions
        for from_state, transitions in self.transitions.items():
            for symbol, to_state in transitions.items():
                dot.edge(from_state, to_state, label=symbol)

        output_path = dot.render(output_file)
        print(f"DFA visualization saved to {output_path}")
    def to_json(self, filename):
        dfa_json = {
            "startingState": self.startingState,
            "acceptingStates": list(self.acceptingStates),
            "transitions": self.transitions
        }
        with open(filename, "w") as f:
            json.dump(dfa_json, f, indent=4)
        print(f"DFA saved to {filename}")
class NFAtoDFAConverter:
    def epsilon_closure(self,NFA,state):
        closure = {state}
        stack = [state]

        while stack:
            current = stack.pop()

            if "ε" in NFA[current]:
                for target in NFA[current]["ε"]:
                    if target not in closure:
                        closure.add(target)
                        stack.append(target)

        return closure
    def convert_to_DFA(self,NFA):
        
        DFA_machine = DFA()
        # Step 1: Compute the starting state (ε-closure of NFA start)
        start_closure = self.epsilon_closure(NFA, NFA["startingState"])
        start_state = State("S1", start_closure)
        start_state.elements=frozenset(start_state.elements)
        DFA_machine.add_state(start_state, self.is_accepting_state(NFA, start_closure))
        DFA_machine.startingState = start_state.name

        # Step 2: Process unmarked states
        unmarked_states = [start_state]

        while unmarked_states:
            current_state = unmarked_states.pop()

            # Step 3: Iterate over all input symbols (exclude ε)
            alphabet = self.get_alphabet(NFA)

            for symbol in alphabet:
                next_states = set()

                for nfa_state in current_state.elements:
                    if symbol in NFA[nfa_state]:
                        for target in NFA[nfa_state][symbol]:
                            next_states |= self.epsilon_closure(NFA, target)

                if not next_states:
                    continue

                next_state = State(f"S{len(DFA_machine.states)}", next_states)

                if next_state not in DFA_machine.states.values():
                    DFA_machine.add_state(next_state, self.is_accepting_state(NFA, next_states))
                    unmarked_states.append(next_state)
                else:
                    # Find the existing state
                    next_state = next(s for s in DFA_machine.states.values() if s == next_state)

                # Add the transition
                DFA_machine.add_transition(current_state, next_state, symbol)

        return DFA_machine
    def is_accepting_state(self, NFA, state_set):
        return any(NFA[s].get("isTerminatingState", False) for s in state_set)


    def get_alphabet(self, NFA):
        alphabet = set()
        for transitions in NFA.values():
            if(not isinstance(transitions,str)):
                for symbol in transitions.keys():
                    if symbol != "ε" and symbol != "isTerminatingState":
                        alphabet.add(symbol)
        return alphabet


In [9]:
converter = NFAtoDFAConverter()
dfa = converter.convert_to_DFA(nfa_dict)
dfa.visualize("dfa")
dfa.to_json("dfa_out.json")
# Output DFA structure
print("Starting State:", dfa.startingState)
print("Accepting States:", dfa.acceptingStates)
print("Transitions:")
for from_state, transitions in dfa.transitions.items():
    for symbol, to_state in transitions.items():
        print(f"{from_state} --{symbol}--> {to_state}")


DFA visualization saved to dfa.png
DFA saved to dfa_out.json
Starting State: S1
Accepting States: {'S1'}
Transitions:
S1 --a--> S1
